In [1]:
import pandas as pd
import math
import ipywidgets as widgets

from sidepanel import SidePanel

import regulus
from ipyregulus import GraphView, AxesCtrl, DetailsView, TreeView, ProjectionView, AttrFilter
from ipyregulus.core.axis import Axis
from ipyregulus.utils import *

### Data

In [2]:
name = 'gauss4'
data = regulus.load(name)

In [3]:
simplified = regulus.SimplifiedTree(data.tree, AttrFilter(attr='span'))

In [4]:
simplified.filter.value = -1

## Graph View

In [5]:
g = GraphView(tree=simplified, show_inverse=False)
g.color = 'z'
gp = SidePanel((g, 'graph'), title='GraphView')

#### Control axis size 

In [6]:
ctrl = AxesCtrl()
widgets.link((g, 'axes'), (ctrl, 'axes'))
ctrl

AxesCtrl(children=(HBox(children=(Checkbox(value=False), UnboundedFloatSlider(value=200.0, description='x1', m…

#### ... or add to the Graph Panel

In [7]:
gp.add(ctrl, header='Axes', expand=False)

### Tree View

In [8]:
tv = TreeView(simplified)
tp = SidePanel([(tv, 'tree'), (simplified.filter, 'Simplify')], title='TreeView')
widgets.link((tv.view, 'details'), (g,'show'));

In [9]:
dv = DetailsView(data=data)
widgets.link((tv.view, 'details'), (dv, 'show'));
widgets.link((tv.view, 'highlight'), (dv, 'highlight'));

dp = SidePanel(title='Details')
dp.add(dv)

In [10]:
# dv.close()

#### _Select partitions_ 

### Let's select all partitions at a given persistence

In [11]:
def at_persistence(v):
    g.show = [ p.id for p in data.tree.at_persistence(v)]

In [12]:
ps = widgets.FloatSlider(min=0, max=1, step=0.001, value=0.5)

In [13]:
widgets.interact(at_persistence, v=ps);

interactive(children=(FloatSlider(value=0.5, description='v', max=1.0, step=0.001), Output()), _dom_classes=('…

#### ... or add the slider to the Graph Panel

In [13]:
gp.add(ps, header='Persistence', expand=False)

### Show inverse curves

In [10]:
cb = widgets.Checkbox(
    value=False,
    description='Show Inverse Curves',
    disabled=False
)
widgets.link((g, 'show_inverse'), (cb, 'value'))
gp.insert(0, cb, expand=False)

## Project the points in the selected partitions

In [11]:
pr = ProjectionView(data.tree)
pr.color = 'y'
prp = SidePanel((pr, 'Projection'), title='ProjectionView', side='split-bottom', ref='TreeView')
widgets.link((g, 'show'), (pr, 'show'));

#### Coordiante the axes of the Graph and Projection views

In [12]:
widgets.link((g, 'axes'), (pr, 'axes'));

### set color based on the pt's partition 

In [18]:
def partition_color(pv):
    tree = pv.tree
    partitions = tree.find_partitions(pv.show)
    
    def in_partition(idx):
        for p in partitions:
            if idx in p.idx:
                return p.id
        return -1
    
    colors = list(map(in_partition, pv._index))
    return colors

In [19]:
pr.color = partition_color

In [20]:
pr.color = 'y'